# Generate QnA synthetic dataset from a Complex PDF using Azure AI Document Intelligence

### Overview

We process the PDF by dividing it into three parts.

-   **Text-heavy** - Text-heavy PDF can be processed with open source without the need to use toolkits like Azure AI Document Intelligence.
-   **Image-heavy** - Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.
-   **Mixed** - After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)

![summary](../imgs/summary-creating-qna-pdf.png)


In [1]:
%load_ext autoreload
%autoreload 2 
# auto-reload all the modules. Great way to make writing and testing your modules much easier. Refer to: https://nocomplexity.com/documents/jupyterlab/tip-autoloadmodule.html
import os, sys
lab_prep_dir = os.getcwd().split("SLMWorkshopCN")[0] + "SLMWorkshopCN/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

Kernel: pythonundefinedundefinedundefinedjvsc74a57bd02139c70ac98f3202d028164a545621647e07f47fd6f5d8ac55cf952bf7c15ed1


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()#相关机密配置写在.env中，并.gitignore掉.env

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")
    
print(f"aoai_api_endpoint: {aoai_api_endpoint}")
# print(f"aoai_api_key: {aoai_api_key}")
print(f"aoai_api_version: {aoai_api_version}")
print(f"aoai_deployment_name: {aoai_deployment_name}")
print(f"doc_intelligence_endpoint: {doc_intelligence_endpoint}")
# print(f"doc_intelligence_key: {doc_intelligence_key}")

aoai_api_endpoint: https://wstus2aisrv.cognitiveservices.azure.com/
aoai_api_version: 2024-08-01-preview
aoai_deployment_name: gpt-4o
doc_intelligence_endpoint: https://wstus2docintel.cognitiveservices.azure.com/


## 1. Read & Preprocess PDF file

---


### Split the PDFs into individual pages


In [3]:
import shutil, random
import openai
from langchain_community.document_loaders.csv_loader import CSVLoader
from util.common_utils import get_language_code

raw_data_dir = "../raw_data"
splitted_raw_data_dir = "splitted_raw_data"
file_path = f"{raw_data_dir}/pdf/en-imagenet-training-wrote-by-daekeun.pdf"

DOMAIN = "Distributed training on Cloud"
LANGUAGE = "English" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE) # 用于找到不同语言的prompt_template
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

Domain: Distributed training on Cloud, Language: English, Language Code: en


(Optional) Only use a poration of the PDF documents for testing. If there are a lot of pages or partial processing is required, cut and save only some pages.


In [4]:
import fitz #PyMuPDF

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(5, 25)] #第5页到最后

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/part{idx}.pdf")

Distinguish between pages composed mainly of text, pages composed primarily of images, and pages composed of mixed text/images.


In [5]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import analyze_pdf_page_content, split_pdf

analyzed_pdf_result = analyze_pdf_page_content(file_path)#判断Pdf每页的类型:Text,Image,Mixed，及其对应页码
delete_folder_and_make_folder(splitted_raw_data_dir)    

print("### PDF Content Analysis Result:")
for content_type, pages in analyzed_pdf_result.items():
    print(f"{content_type} pages: {pages}")
    split_pdf(file_path, f"{splitted_raw_data_dir}/{content_type}.pdf", pages)#按Pdf每页的类型保存中间结果供后续使用
print(splitted_raw_data_dir)

### PDF Content Analysis Result:
Text pages: [0, 1, 3, 5, 8]
Mixed pages: [2, 4, 6, 7]
splitted_raw_data


In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat
from openai import AzureOpenAI

try:
    client = AzureOpenAI(  # 构造azure openai client
        api_key=aoai_api_key,  
        api_version=aoai_api_version,
        base_url=f"{aoai_api_endpoint}/openai/deployments/{aoai_deployment_name}",
        max_retries=1
    )
except (ValueError, TypeError) as e:
    print(e)

try:
    document_intelligence_client = DocumentIntelligenceClient(  # 构造azure doc intelligence client. 参考：https://github.com/microsoft/Form-Recognizer-Toolkit/blob/main/SampleCode/Python/sample_figure_understanding.ipynb
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )
except (ValueError, TypeError) as e:
    print(e)    

### Case 1: Mixed page (Images and text mixed appropriately)

After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)


#### Analyze Document

Analyze the document with Azure AI Document Intelligence to extract the text and image information.

-   `crop_image_from_file()`: Crop the image from the PDF file based on the bounding box.
-   `is_bounding_box_larger_than()`: Check if the bounding box is larger than the threshold.
-   `image_complexity()`: Check if the image is complex or simple based on image statistics.
-   `understand_image_with_gpt():` Summarize the image with OpenAI GPT.

![post-process](../imgs/post-process1.png)
![post-process](../imgs/post-process2.png)


In [7]:
if "Mixed" in analyzed_pdf_result:
    pdf_mixed_path = f"{splitted_raw_data_dir}/Mixed.pdf"

    with open(pdf_mixed_path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(# layout分析的用法参考 https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/prebuilt/layout?view=doc-intel-4.0.0&tabs=sample-code
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
            output_content_format=ContentFormat.MARKDOWN 
        )

    result = poller.result()# azure dock intelligence返回的对象包含很多有用内容，参考：https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept/analyze-document-response?view=doc-intel-3.1.0&source=recommendations
    md_content = result.content

    #### Updates the content of the figure description (empty content or caption) with the image summary text generated by gpt-4o.
    from util.preprocess import (
        image_complexity, is_bounding_box_larger_than, crop_image_from_file, 
        understand_image_with_gpt, update_figure_description
    )
    output_folder = "pdf_mixed_tmp"
    delete_folder_and_make_folder(output_folder)
    language = LANGUAGE
    max_tokens = 1024
    input_file_path = file_path

    if result.figures:
        print("Figures:")
        for idx, figure in enumerate(result.figures): # 处理azure doc intelligence找到的每一张图片
            img_description = ""
            #print(f"Figure #{idx} has the following spans: {figure.spans}")

            # figure_content = ""
            # for i, span in enumerate(figure.spans):# span: details the text spans related to the figure, specifying their offsets and lengths within the document's text. This connection helps in associating the figure with its relevant textual context
            #     #print(f"Span #{i}: {span}")
            #     figure_content += md_content[span.offset:span.offset + span.length] #图片在md中的位置，起始字符下表和结束字符下标
            # #print(f"Original figure content in markdown: {figure_content}")

            # Note: figure bounding regions currently contain both the bounding region of figure caption and figure body
            if figure.caption:# azure doc intelligence分析出来的figure caption是pdf上图片附近的那一小串说明，不一定有
                caption_region = figure.caption.bounding_regions
                #print(f"\tCaption: {figure.caption.content}")
                #print(f"\tCaption bounding region: {caption_region}")
                for region in figure.bounding_regions:# Each region specifies the page number (1-indexed) and bounding polygon. The bounding polygon is described as a sequence of points, clockwise from the left relative to the natural orientation of the element. For quadrilaterals, plot points are top-left, top-right, bottom-right, and bottom-left corners. Each point represents its x, y coordinate in the page unit specified by the unit property. In general, unit of measure for images is pixels while PDFs use inches.
                    if region not in caption_region:# 如果图片不是只有caption的话，则需要处理其它真正的图片
                        #print(f"\tFigure body bounding regions: {region}")
                        # To learn more about bounding regions, see https://aka.ms/bounding-region
                        boundingbox = (
                                region.polygon[0],  # x0 (left)
                                region.polygon[1],  # y0 (top)
                                region.polygon[4],  # x1 (right)
                                region.polygon[5]   # y1 (bottom)
                            )

                        if is_bounding_box_larger_than(boundingbox):# 图像足够大(超1x1英寸)才会截取出来处理
                            #print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")
                            cropped_image = crop_image_from_file(pdf_mixed_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                            if image_complexity(cropped_image)[0] == "Complex":# 图片的熵，清晰度，边缘数都超过阈值，则定义为复杂的，信息量大的，有意义的图片，后续需要处理
                                # Get the base name of the file
                                base_name = os.path.basename(input_file_path)
                                # Remove the file extension
                                file_name_without_extension = os.path.splitext(base_name)[0]

                                output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                                cropped_image_filename = os.path.join(output_folder, output_file)

                                cropped_image.save(cropped_image_filename)
                                print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")

                                try: # 用gpt-4o生成图片描述. 如果azure doc intellegence在图片附近找到了caption，则作为上下文传入
                                    image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, caption, max_tokens=max_tokens, language=language)
                                except openai.BadRequestError as e:
                                    print(f"BadRequestError: {e}")
                                    image_summarization = ""
                                img_description += image_summarization

                                print(f"\tDescription of figure {idx}: {img_description}")
                            else:
                                print(f'simple image at idx {idx}')

            else:# 如果pdf上图片附近找不到相关caption说明
                #print("\tNo caption found for this figure.")
                for region in figure.bounding_regions:
                    #print(f"\tFigure body bounding regions: {region}")
                    # To learn more about bounding regions, see https://aka.ms/bounding-region
                    boundingbox = (
                            region.polygon[0],  # x0 (left)
                            region.polygon[1],  # y0 (top
                            region.polygon[4],  # x1 (right)
                            region.polygon[5]   # y1 (bottom)
                        )

                    if is_bounding_box_larger_than(boundingbox):# 图像足够大(超1x1英寸)才会截取出来处理
                        #print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")

                        cropped_image = crop_image_from_file(input_file_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                        if image_complexity(cropped_image)[0] == "Complex":# 图片的熵，清晰度，边缘数都超过阈值，则定义为复杂的，信息量大的，有意义的图片，后续需要处理
                            # Get the base name of the file
                            base_name = os.path.basename(input_file_path)
                            # Remove the file extension
                            file_name_without_extension = os.path.splitext(base_name)[0]

                            output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                            cropped_image_filename = os.path.join(output_folder, output_file)
                            # cropped_image_filename = f"data/cropped/image_{idx}.png"
                            cropped_image.save(cropped_image_filename)
                            #print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")

                            try: # 用gpt-4o生成图片描述
                                image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, "", max_tokens=max_tokens, language=language)
                            except openai.BadRequestError as e:
                                print(f"BadRequestError: {e}")
                                image_summarization = ""
                            img_description += image_summarization
                            print(f"\tDescription of figure {idx}: {img_description}")
                        else:
                            print(f'simple image at idx {idx}')

            
            md_content = update_figure_description(md_content, img_description, idx) # 把azure gpt-4o生成的图片描述插入到原文档中放置图片(图片占位符)的地方，此时返回的整个文档md_content在原来有图片的地方就包含了图片描述

Figures:
5.958200000000001 3.0484
	Description of figure 0: The image is a screenshot of a text box containing a sequence of shell commands to download the ImageNet dataset. Below it, there is a heading for an alternative method if the first method does not work.

The shell commands in the text box are:
```
$ export IMAGENET_USERNAME=[YOUR_USERNAME]
$ export IMAGENET_ACCESS_KEY=[YOUR_ACCESS_KEY]
$ cd imagenet/data
$ mv imagenet_2012_validation_synset_labels.txt synsets.txt
$ nohup bash download_imagenet.sh synsets.txt >& download.log &
```


Generate chunks for mixed pages


In [8]:
if "Mixed" in analyzed_pdf_result:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import re

    text_splitter = RecursiveCharacterTextSplitter(# 按分隔符优先级递归对文档进行分隔成chunk，也可考虑azure doc intelligence的
        separators=[
            r'<!-- PageNumber="\d+" -->', # azure doc intelligence 的页分隔符
            r"\n\n",
            r"\n",
            " ",
            ".",
            "",
        ],   
        is_separator_regex = True,    
        chunk_size=2000,
        chunk_overlap=200,
    )

    mixed_chunks = text_splitter.split_text(md_content) # 按照上述页，章节，段的分隔符把原文档分割成多个chunk
else:
    mixed_chunks = []
print("Length of splits (mixed case): " + str(len(mixed_chunks)))

Length of splits (mixed case): 8


### Case 2: Text-heavy



In [9]:
if "Text" in analyzed_pdf_result:
    from langchain_community.document_loaders.pdf import PyMuPDFLoader
    from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

    pdf_text_path = f"{splitted_raw_data_dir}/Text.pdf" # 加载在‘Split the PDFs into individual pages’中分割出来的文字为主的pdf页
    loader = PyMuPDFLoader(pdf_text_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, 
        chunk_overlap=200
    )

    text_chunks = text_splitter.split_documents(documents)

    for idx, chunk in enumerate(text_chunks):
        print(f"Chunk {idx}\n{chunk}")
        print("="*80)
        if idx == 2:
            break

    text_chunks = [d.page_content for d in text_chunks]
else:
    text_chunks = []
print("Length of splits (text-heay case): " + str(len(text_chunks)))

Chunk 0
page_content='[Hands-on] Fast Training
ImageNet on on-demand EC2
GPU instances with Horovod
💻
Author: Daekeun Kim (daekeun@amazon.com)
Goal
This document is for people who need distributed GPU training using Horovod for
experimental purposes. Many steps are similar to what mentioned in Julien
Simon’s article(
) and AWS
Documentation(
). So I recommend you to view these articles first. If there
are some things that aren’t going well (e.g., Downloading the dataset does not
work, How to convert the raw data to the TFRecord feature set?, How to fix the
error ModuleNotFoundError: No module named 'cv2'? ) please refer this
document.
https://medium.com/@julsimon/imagenet-part-1-going-on-an-
adventure-c0a62976dc72
https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-
horovod-tensorflow.html
Introduction
For data preparation and data transformation, we do not need to use a GPU
instance such as p2 and p3. Instead, we can start much cheaper instances like 
t2.large  instance with 1.

### Case 3: Image-heavy

Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.

### Preprocess Image


In [10]:
if "Image" in analyzed_pdf_result:
    import fitz
    from glob import glob

    image_dir = "./pdf_image_tmp"
    delete_folder_and_make_folder(image_dir) 

    pdf_image_path = f"{splitted_raw_data_dir}/Image.pdf" # 加载在‘Split the PDFs into individual pages’中分割出来的图片为主的pdf页
    doc = fitz.open(pdf_image_path)
    #clip_x, clip_y = 10, 45
    clip_x, clip_y = 10, 10 # 基于fitz做的一些矫正

    for i, page in enumerate(doc):
        x, y, w, h = page.rect
        clip = fitz.Rect(x+clip_x, y+clip_y, w-clip_x, h-clip_y)
        page.set_cropbox(clip)
        pix = page.get_pixmap()
        pix.save(f"{image_dir}/page_{i:03d}.jpg")

    images = sorted(glob(os.path.join(image_dir, "*.jpg")))

In [11]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI
# 对‘从图片为主的pdf页’抠出来的图片，用azure gpt-4o进行图像描述
max_tokens = 1024
llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=max_tokens,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name
)

human_prompt_main = f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."

system_prompt = "You are an assistant tasked with describing image."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": human_prompt_main
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()

In [12]:
%%time
if "Image" in analyzed_pdf_result:
    from util.preprocess import encode_image_base64
    #images = glob(os.path.join(image_path, "*.jpg"))
    base64_images = [encode_image_base64(img_path) for img_path in images]
    image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 8})
    image_summaries = remove_short_sentences(image_summaries)
else:
    image_summaries = []
print("Length of image_summaries (image-heavy case): " + str(len(image_summaries)))

Length of image_summaries (image-heavy case): 0
CPU times: user 242 μs, sys: 43 μs, total: 285 μs
Wall time: 253 μs


## 2. Construct QnA Pairs

---

### Option 1.

Leverage the `azure-ai-generative` package. The `QADataGenerator` class in this package makes it easy to generate QnA synthetic questions. However, using this class as is has the disadvantage of not being able to use custom prompts, so we inherited from it and created the `CustomQADataGenerator` class.


In [13]:
from util.qa import CustomQADataGenerator # 继承自azure-ai-generative的QADataGenerator，封装了如何获取自己的prompt template的逻辑
model_config = {
    "deployment": aoai_deployment_name,
    "model": "gpt-4o",
    "max_tokens": 2000,
}

qa_generator = CustomQADataGenerator(model_config=model_config, templates_dir=f"./prompt_template/{LANGUAGE_CODE}")

In [14]:
import asyncio
from collections import Counter
from typing import Dict
import os
from azure.ai.generative.synthetic.qa import QAType
concurrency = 6  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

#qa_type = QAType.CONVERSATION
qa_type = QAType.LONG_ANSWER

async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(# 根据 QAType.LONG_ANSWER 和 LANGUAGE_CODE 会获取到提示词模板 1_synthetic-qa-generation/seed/prompt_template/en/prompt_qa_long_answer.txt
            text=text,
            qa_type=qa_type,
            num_questions=3,  # Number of questions to generate per text
        )

In [15]:
input_batch = mixed_chunks + text_chunks + image_summaries # 有图也有文字的pdf页，文字为主的pdf页，图片为主的pdf页，这些页面的内容都输入让azure gpt-4o生成相关的QA数据
results = await asyncio.gather(*[generate_async(text) for text in input_batch], return_exceptions=True)

question_answer_list = []
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])

print("Successfully generated QAs")

Successfully generated QAs


### Option 2.

You write the entire sequence of code to create a QnA dataset without using azure-ai-generative but azure openai only.


In [16]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair
# 这里演示一下json格式的生成能力
llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template(LANGUAGE) # 另外一种prompt template，包含了context，domain和num_questions变量，并且返回json

chain = prompt | llm | parser

In [17]:
input_batch = []

for doc in mixed_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in text_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in image_summaries:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)        

In [18]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})
print(qa_pair)

[[{'QUESTION': 'What is the recommended Python version to use for TensorFlow models repository according to the context?', 'ANSWER': 'The recommended Python version to use for TensorFlow models repository is Python2 because many codes of TensorFlow models repository do not work on Python3.'}, {'QUESTION': 'Why is it important to install OpenCV when working with ImageNet raw data?', 'ANSWER': 'It is important to install OpenCV because im2rec.py utilizes some OpenCV functions to convert ImageNet raw data to RecordIO files.'}, {'QUESTION': 'Where can you find instructions on how to mount an EBS volume on an EC2 instance?', 'ANSWER': 'Instructions on how to mount an EBS volume on an EC2 instance can be found at https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using-volumes.html.'}], [{'QUESTION': 'What is the first step to download ImageNet according to the provided method?', 'ANSWER': 'The first step to download ImageNet according to the provided method is to go to http://www.image

## 3. Save to jsonl

---

If you want to augment dataset, you can try Evovle-Instruct or other data augmentation techniques.<br>
Please refer to `../evolve-instruct` and `../glan-instruct` for more details.


In [19]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "imagenet-training-summary"
oai_qa_pair = convert_to_oai_format(question_answer_list, system_prompt_msg=system_prompt_msg)

# save_result_jsonl = f"{output_dir}/{save_filename}.jsonl" # option 2
save_result_jsonl = f"{output_dir}/{save_filename}-oai.jsonl" # option 1
#save_jsonl(qa_pair, save_result_jsonl) # option 2
save_jsonl(oai_qa_pair, save_result_jsonl) # option 1
print(f"Saved to {save_result_jsonl}")

Saved to ./dataset/imagenet-training-summary-oai.jsonl


### Clean up


In [20]:
!rm -rf {splitted_raw_data_dir} pdf_image_tmp pdf_mixed_tmp outputs_tmp images

# More Synthetic Data

---

## Evolv-Instruct to generate more

To generate more data from above seed data, we use [Evolve-Instruct](../evolve-instruct/README.md).
![Evolv-Instruct](1_synthetic-qa-generation/evolve-instruct/evlvinstrc.png)

## EvolvMCTS4RL for RL

To improve reasoning capability using RL (Reinforcement Learning), we use [EvolveMCTS4RL](../reasoningplaning/README.md) to generate data.


![EvolvMCTS4RL](../reasoningplaning/evolveMCTS4RL.png)